In [ ]:
import sys
sys.path.insert(0, '../')

import numpy as np
import matplotlib.pyplot as plt

from tilted_stable_dist.rand_exp_tilted_stable import ExpTiltedStableDist

In [ ]:
import rpy2.robjects as robjects
import rpy2.robjects.packages as rpackages

try:
    copula = rpackages.importr('copula')
except:
    utils = rpackages.importr('utils')
    utils.install_packages('copula')
    copula = rpackages.importr('copula')

## Check that outputs from Cythoinized and pure Python codes agree

In [ ]:
from tilted_stable_dist.py_rand_exp_tilted_stable \
    import ExpTiltedStableDist as PyExpTiltedStableDist

In [ ]:
bridge_exponent = .25
char_exponent = bridge_exponent / 2
tilts = 10 ** - np.linspace(-4, 4, 101)
seed = 0

for method in ['double-rejection', 'divide-conquer']:

    tilted_stable = ExpTiltedStableDist(seed)
    cython_samples = tilted_stable.sample(
        char_exponent, tilts, method=method
    )
    py_tilted_stable = PyExpTiltedStableDist(seed=0)
    python_samples = np.array([
        py_tilted_stable.rv(char_exponent, tilt, method=method)
        for tilt in tilts
    ])
    assert np.allclose(cython_samples, python_samples, rtol=10e-12, atol=10e-12)

print("Test passed!")

## Cross check Python module outputs with R package
Generating the half-million samples via an external R call takes about 45 sec on iMac 2015.

In [ ]:
def comparison_hist(samples_1, samples_2, use_log_scale=False):
    if use_log_scale:
        samples_1 = np.log(samples_1)
        samples_2 = np.log(samples_2)
    x_max = max(samples_1.max(), samples_2.max())
    x_min = min(samples_1.min(), samples_2.min())
    bins = np.linspace(x_min, x_max, 51)
    
    plt.hist(samples_1, alpha=.5, bins=bins, density=True)
    plt.hist(samples_2, alpha=.5, bins=bins, density=True)
    for side in ['left', 'top', 'right']:
        plt.gca().spines[side].set_visible(False)
    plt.yticks([])

In [ ]:
plt.figure(figsize=(14, 4.5))
plt.rcParams['font.size'] = 20

n_samples = 5 * 10 ** 5
char_exponent = [1. / 32, 1. / 4]
tilt = [.01, 100.]
tilted_stable = ExpTiltedStableDist()

for i in range(2):
    
    python_samples = tilted_stable.sample(
        char_exponent[i], np.tile(tilt[i], n_samples)
    )
    R_samples = np.array([
        copula.retstable(char_exponent[i], 1.0, tilt[i])[0]
        for _ in range(n_samples)
    ])
    
    plt.subplot(1, 2, i + 1)
    comparison_hist(R_samples, python_samples, use_log_scale=True)
    
    plt.xlabel('log(tilted stable)')
    if i == 0:
        plt.legend(['from R package', 'from Python'], frameon=False)

plt.show()